In [27]:
import numpy as np
import pandas as pd
import re

In [4]:
text = "That person wears marvelous trousers."
pattern = 'er'
# pattern = '[er]'
re.findall(pattern, text)

['er', 'er']

In [5]:
text = "Is it spelled gray or grey?"
# text = "Is it spelled gry or grey?"
pattern = 'gr[ae]y'
re.findall(pattern, text)

['gray', 'grey']

In [35]:
text = '''TKerraPower, A nuclear-energy company founded by Bill Gates,
        is unlikely to follow through on building a demonstration reactor in China,
        due largely to the Trump administration�s crackdown on the country'''

# pattern = '[A-Z][a-z]*'
pattern = '[A-z]+'
# pattern = '[A-Z][a-z]+'
print(re.findall(pattern, text))

['TKerraPower', 'A', 'nuclear', 'energy', 'company', 'founded', 'by', 'Bill', 'Gates', 'is', 'unlikely', 'to', 'follow', 'through', 'on', 'building', 'a', 'demonstration', 'reactor', 'in', 'China', 'due', 'largely', 'to', 'the', 'Trump', 'administration', 's', 'crackdown', 'on', 'the', 'country']


### More complex regex

- `\w`: Any alphanumeric character.
- `\W`: Any non-alphanumeric character.
- `\d`: Any numeric character.
- `\D`: Any non-numeric character.
- `\s`: Any whitespace characters.
- `\S`: Any non-whitespace characters.

In [15]:
text = "If you tell the truth, you don't have to remember anything 100."

# pattern = '\w'
# pattern = '\w+'
# pattern = '\w*'
# pattern = '\w?'
pattern = '.'
print(re.findall(pattern, text))

['I', 'f', ' ', 'y', 'o', 'u', ' ', 't', 'e', 'l', 'l', ' ', 't', 'h', 'e', ' ', 't', 'r', 'u', 't', 'h', ',', ' ', 'y', 'o', 'u', ' ', 'd', 'o', 'n', "'", 't', ' ', 'h', 'a', 'v', 'e', ' ', 't', 'o', ' ', 'r', 'e', 'm', 'e', 'm', 'b', 'e', 'r', ' ', 'a', 'n', 'y', 't', 'h', 'i', 'n', 'g', ' ', '1', '0', '0', '.']


In [17]:
text = "If you tell the truth, you don't have to remember anything 100."
pattern = '\w{4}'
# pattern = '\w{4,}'
print(re.findall(pattern, text))

['tell', 'trut', 'have', 'reme', 'mber', 'anyt', 'hing']


In [18]:
text = """
Aeromexico 800 -237- 6639
Air Canada 888-247-2262
Air Canada Rouge 888-247-2262
Air Creebec 800-567-6567
Air Inuit 800-361-2965
Air North 800-661-0407
Air Tindi 888-545-6794"""
pattern = '\d+-\d+-\d+'
re.findall(pattern, text)

['888-247-2262',
 '888-247-2262',
 '800-567-6567',
 '800-361-2965',
 '800-661-0407',
 '888-545-6794']

### Feature Engineering

- **Feature engineering** is a process of transforming the given data into a form that is easier to interpret. All the techniques that we have used before including data transformations techniques, encoding categorical variables, scaling numerical features, imputing missing values, cleaning categorical columns, using regular expressions, using DateTime, and string functions are feature engineering techniques. The key idea is that we are manipulating the information that is available to us to be able to better understand it and improve the model.

- **Feature extraction/feature generation**: It is the process of extracting relevant information from the existing available information. For example, you are provided with the date of birth of customers in the data. You are not interested in when they were born but more interested in their age. In that case, you can use the DateTime functions to calculate the age or extract other relevant information such as the year they were born, the month they were born.


- **Feature selection**: This is the process of selecting the features/columns in your data that are relevant to the model, for eg. if it is a prediction problem, you will be more interested in only those variable that have an impact on your target and not other columns/features that are not providing any information about target. Adding variables that don't add value to the model degrades the performance of the model both in terms of accuracy and in terms of efficiency.

Some of the feature selection methods that we took a look at earlier include:

- Checking null values to drop a column
- Sense check to drop columns that are not significant
- Using heat maps to check multicollinearity for numerical variables
- Chi-square tests for categorical variables

In [21]:
data = pd.read_csv('4.01_healthcare.csv')

In [24]:
data.shape

(90569, 28)

In [23]:
len(data[data['DOB']==0])

22507

In [25]:
def year(x):
    x = str(x)
    if len(x)<4:
        return np.nan
    else:
        pattern = '\d\d'
        yr = re.findall(pattern,x)[0]
        return int(yr)


Note that our reference year is 1997 here as the data is from that study.


In [28]:
data['year'] = list(map(year, data['DOB']))
data['year'] = 97 - data['year']

data['year'] = data['year'].fillna(np.mean(data['year']))

# Now we can drop the column DOB as we have extracted the information we need from this column
data = data.drop(['DOB'], axis=1)

## Multicollinearity

Through multivariate linear regression, we are trying to assess the influence of each of the predictor variables on the target variable. This influence/relationship is linear and is represented by a mathematical equation. The equation is given as: `Y=β0+β1X1+β2X2+β3X3+β4X4+........+βnXn`.
Here, each variable is trying to explain some information about the nature of `Y`, how does `Y` change with each of the predictor variables. The change in `Y` with `X` is technically variation. When the predictor variables are all independent of each other, each variable explains some information on the change in `Y`. Multicollinearity arises when the predictor variables are highly correlated. Hence some predictors are redundant as they do not reveal any new information on the change in `Y` with a change in `X`.

The correlation matrix using the heat maps helps us understand the correlation between the independent variables.
Using the `sklearn.metrics` module we calculate `R` square statistic. It measures the proportion of variance in the dependent variable that is explained by all of the independent variables.

For checking multicollinearity, we calculate `R` square `k` and `VIF` (variance inflation factor) for each of the `k` independent variables. We do this by regressing the `k`-th independent variable on all of the other independent variables. That is, we treat `X` `k` as the dependent variable and use the other independent variables to predict `X` `k`.

For eg. `Y=β0+β1X1+β2X2+β3X3+β4X4`

Build a model `X1` vs. `X2 X3 X4`, find `R^2`, call it `R1`.
Build a model `X2` vs. `X1 X3 X4`, find `R^2`, call it `R2`.
and so on and so forth

**Interpreting R square k** - If `R2k `equals zero, variable `k` is not correlated with any other independent variable.
Usually, multicollinearity is a potential problem when `R2k` is greater than `0.75` and, a serious problem when `R2k` is greater than `0.9`.

For each variable that we find individual R2, `VIF = 1 / ( 1 - R2k )` (for each dependent variable `k`). It is used to assess multicollinearity.

**Interpretation of the variance inflation factor**: If `VIFk = 1`, variable k is not correlated with any other independent variable. Multicollinearity is a potential problem when `VIFk` is greater than 4 and, a serious problem when it is greater than 10.

### Effects of Multicollinearity:

1. It makes it harder to interpret the significance of variables in the regression model (we will talk about statistical significance/p-value later).
2. It might give good enough results due to over-fitting, but those will not be very reliable (we will talk about over-fitting and under-fitting in more detail later. Over-fitting for now, you can explain to the students as a more complicated model and not very generalized ie it might work on the data at hand very well but not so well on the unknown/out of the box data.
3. It is also important to note that it does not severely impact the model in terms of predicting power if the only prediction is the main goal of the analysis.

Dealing with High Multicollinearity:

1. Centering/standardizing/normalizing variables may help reduce multicollinearity.
2. Removing one or more of the variables that are highly correlated with each other.


## VIF (Variance Inflation Factor)
Additional resource [here](https://etav.github.io/python/vif_factor_python.html)

In [32]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# This is the code to show how to use the __variance_inflation_factor__ function
# We are using add_constant here as VIF method in python expects the addition of a constant terms in the X features. It uses OLS but does not add the constant itself.
# data_corr = data.corr()

vif = {}
data_corr = add_constant(data_corr)
for i in np.arange(data_corr.shape[1]):
    column_name = data_corr.columns[i]
    value = variance_inflation_factor(np.array(data_corr), i)
    vif[column_name] = value

# Code to use the variance_inflation_factor technique to remove highly correlated columns

flag = True
threshold = 50
data_corr = add_constant(data_corr)
while flag is True:
    #print(data_corr.head())
    flag = False
    values = [variance_inflation_factor(np.array(data_corr), i) for i in np.arange(data_corr.shape[1])]
    #print(values)
    if max(values)> threshold:
        col_index = values.index(max(values))
        column_name = data_corr.columns[col_index]
        data_corr = data_corr.drop([column_name], axis=1)
        flag = True


/Users/abhimanyumittal/venv/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/abhimanyumittal/venv/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [33]:
vif

{'const': 0.0,
 'INCOME': inf,
 'HV1': inf,
 'HV2': inf,
 'HV3': inf,
 'HV4': inf,
 'IC1': inf,
 'IC2': inf,
 'IC3': inf,
 'IC4': inf,
 'IC5': inf,
 'NUMPROM': inf,
 'CARDPROM': inf,
 'CARDPM12': inf,
 'NUMPRM12': inf,
 'MAXADATE': inf,
 'NGIFTALL': inf,
 'TIMELAG': inf,
 'AVGGIFT': inf,
 'year': inf}